# Importing Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support

#from scipy import stats
# from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
# from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#import sklearn as skl

# from sklearn.metrics.pairwise import cosine_similarity
# from scipy import sparse

# import tensorflow as tf

# Loading the Data

In [3]:
df_bg_tmp = pd.read_csv('../datasets/games-cleaned.csv')
df_rv_tmp = pd.read_csv('../datasets/reviews-cleaned.csv')

# Content-based Recommenders

In [4]:
df_bg = df_bg_tmp.copy()
df_rv = df_rv_tmp.copy()

In [5]:
df_bg.head()

,id,name,rank,geek_rating,avg_rating,usersrated,category,mechanic,weight,minplayers,maxplayers,playingtime,minage,owned,yearpublished,designer,artist,publisher,description,image
0,30549,Pandemic,106,7.48669,7.58896,109006,['Medical'],"['Action Points', 'Cooperative Game', 'Hand Ma...",2.4063,2,4,45,8,168364,2008,['Matt Leacock'],"['Josh Cappel', 'Christian Hanisch', 'Régis Mo...","['Z-Man Games', 'Albi', 'Asmodee', 'Asmodee It...","In Pandemic, several virulent diseases have br...",https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...
1,822,Carcassonne,191,7.30857,7.41837,108776,"['City Building', 'Medieval', 'Territory Build...","['Area Majority / Influence', 'Map Addition', ...",1.9057,2,5,45,7,161299,2000,['Klaus-Jürgen Wrede'],"['Doris Matthäus', 'Anne Pätzke', 'Chris Quill...","['Hans im Glück', '999 Games', 'Albi', 'Bard C...",Carcassonne is a tile-placement game in which ...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...
2,13,Catan,429,6.96965,7.13598,108064,"['Economic', 'Negotiation']","['Dice Rolling', 'Hexagon Grid', 'Income', 'Mo...",2.3130,3,4,120,10,167733,1995,['Klaus Teuber'],"['Volkan Baga', 'Tanja Donner', 'Pete Fenlon',...","['KOSMOS', '999 Games', 'Albi', 'Asmodee', 'As...","In CATAN (formerly The Settlers of Catan), pla...",https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...
3,68448,7 Wonders,73,7.63355,7.73515,90021,"['Ancient', 'Card Game', 'City Building', 'Civ...","['Drafting', 'Hand Management', 'Set Collectio...",2.3264,2,7,30,10,120466,2010,['Antoine Bauza'],"['Dimitri Chappuis', 'Miguel Coimbra', 'Etienn...","['Repos Production', 'ADC Blackfire Entertainm...",You are the leader of one of the 7 great citie...,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0I...
4,36218,Dominion,104,7.49912,7.61000,81582,"['Card Game', 'Medieval']","['Deck, Bag, and Pool Building', 'Delayed Purc...",2.3542,2,4,30,13,106956,2008,['Donald X. Vaccarino'],"['Matthias Catrein', 'Julien Delval', 'Tomasz ...","['Rio Grande Games', '999 Games', 'Albi', 'Bar...","&quot;You are a monarch, like your parents bef...",https://cf.geekdo-images.com/j6iQpZ4XkemZP07HN...


In [6]:
df_rv.head()

,user,name,rating
0,1 Family Meeple,10 Days in Europe,4.1
1,1 Family Meeple,12 Days,7.0
2,1 Family Meeple,7 Wonders,6.5
3,1 Family Meeple,A Column of Fire,5.0
4,1 Family Meeple,A Feast for Odin,10.0


In [7]:
df_bg_cat = df_bg[['name', 'category', 'mechanic', 'designer', 'artist', 'publisher']].copy()

df_bg_cat = df_bg_cat.set_index('name')
df_bg_cat.head()

,category,mechanic,designer,artist,publisher
name,,,,,
Pandemic,['Medical'],"['Action Points', 'Cooperative Game', 'Hand Ma...",['Matt Leacock'],"['Josh Cappel', 'Christian Hanisch', 'Régis Mo...","['Z-Man Games', 'Albi', 'Asmodee', 'Asmodee It..."
Carcassonne,"['City Building', 'Medieval', 'Territory Build...","['Area Majority / Influence', 'Map Addition', ...",['Klaus-Jürgen Wrede'],"['Doris Matthäus', 'Anne Pätzke', 'Chris Quill...","['Hans im Glück', '999 Games', 'Albi', 'Bard C..."
Catan,"['Economic', 'Negotiation']","['Dice Rolling', 'Hexagon Grid', 'Income', 'Mo...",['Klaus Teuber'],"['Volkan Baga', 'Tanja Donner', 'Pete Fenlon',...","['KOSMOS', '999 Games', 'Albi', 'Asmodee', 'As..."
7 Wonders,"['Ancient', 'Card Game', 'City Building', 'Civ...","['Drafting', 'Hand Management', 'Set Collectio...",['Antoine Bauza'],"['Dimitri Chappuis', 'Miguel Coimbra', 'Etienn...","['Repos Production', 'ADC Blackfire Entertainm..."
Dominion,"['Card Game', 'Medieval']","['Deck, Bag, and Pool Building', 'Delayed Purc...",['Donald X. Vaccarino'],"['Matthias Catrein', 'Julien Delval', 'Tomasz ...","['Rio Grande Games', '999 Games', 'Albi', 'Bar..."


In [8]:
# Defining the map function
def dummies(x,df):
    temp = pd.get_dummies(df[x].str.split(',', expand=True), prefix='', prefix_sep='', drop_first = True).sum(level=0, axis=1)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df
    
df_dummy = df_bg_cat.copy()

for col in df_bg_cat.columns:
    df_dummy[col] = df_dummy[col].str.strip('[]')
    df_dummy[col] = df_dummy[col].str.replace(" ", '')
    df_dummy = dummies(col,df_dummy)
df_dummy.head()

,'AbstractStrategy','Action/Dexterity','Adventure','AgeofReason','AmericanCivilWar','AmericanIndianWars','AmericanRevolutionaryWar','AmericanWest','Ancient','Animals',...,'StoryFactory','Litor','NordicGamesehf','Q-Workshop','LouvreEditions','MeeplesCafe','TempoGames(I)','Rabén&Sjögren','ParadigmInfinitumPteLtd','SunTeamDayHobbyCompany'
name,,,,,,,,,,,,,,,,,,,,,
Pandemic,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Carcassonne,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Catan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7 Wonders,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Dominion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
content_sim_df = pd.DataFrame(cosine_similarity(df_dummy), columns=df_dummy.index, index=df_dummy.index)
content_sim_df.head()

name,Pandemic,Carcassonne,Catan,7 Wonders,Dominion,Ticket to Ride,Codenames,Terraforming Mars,7 Wonders Duel,Agricola,...,RRR,Magic: The Gathering – Heroes of Dominaria Board Game,"Gulf, Mobile & Ohio",1805: Sea of Glory,Steam Donkey,Feierabend,Nations at War: White Star Rising,Oaxaca: Crafts of a Culture,A Gentle Rain,Wizards of the Wild
name,,,,,,,,,,,,,,,,,,,,,
Pandemic,1.000000,0.236573,0.113901,0.385922,0.202567,0.248093,0.241890,0.177954,0.314378,0.300188,...,0.000000,0.046127,0.000000,0.00000,0.087960,0.00000,0.000000,0.000000,0.000000,0.084215
Carcassonne,0.236573,1.000000,0.138159,0.139320,0.364013,0.167183,0.177822,0.179878,0.132407,0.252861,...,0.081786,0.000000,0.046625,0.00000,0.000000,0.00000,0.000000,0.000000,0.060193,0.000000
Catan,0.113901,0.138159,1.000000,0.029514,0.144589,0.088542,0.094176,0.095265,0.000000,0.107134,...,0.000000,0.098773,0.098773,0.08663,0.000000,0.11713,0.055216,0.043315,0.000000,0.045083
7 Wonders,0.385922,0.139320,0.029514,1.000000,0.204124,0.285714,0.170941,0.259376,0.610960,0.162051,...,0.000000,0.000000,0.000000,0.00000,0.170941,0.00000,0.000000,0.052414,0.000000,0.163663
Dominion,0.202567,0.364013,0.144589,0.204124,1.000000,0.174964,0.116311,0.094124,0.138568,0.264628,...,0.000000,0.000000,0.048795,0.00000,0.093048,0.00000,0.000000,0.171184,0.000000,0.044544


In [10]:
# Top 20 similar board games
content_input = 'Wingspan'
content_sim = content_sim_df[content_input]
content_sim = content_sim.sort_values(ascending=False).head(20)
content_sim

name
Wingspan                                  1.000000
Viticulture Essential Edition             0.446026
Tapestry                                  0.438529
Terraforming Mars                         0.415128
Red Rising                                0.393398
Charterstone                              0.376845
Between Two Castles of Mad King Ludwig    0.359546
Everdell                                  0.327086
Between Two Cities                        0.314485
Scythe                                    0.311715
Nevermore                                 0.303822
7 Wonders                                 0.302614
Canvas                                    0.300501
Pendulum                                  0.300501
Patchwork                                 0.289683
7 Wonders Duel                            0.287599
Die Pyramiden des Jaguar                  0.283069
Rolling Realms                            0.280224
Sushi Go!                                 0.278747
Port Royal                

In [11]:
df_bg_cat.loc[content_sim.index[:4]]

,category,mechanic,designer,artist,publisher
name,,,,,
Wingspan,"['Animals', 'Card Game', 'Educational']","['Card Drafting', 'Dice Rolling', 'Drafting', ...",['Elizabeth Hargrave'],"['Ana Maria Martinez Jaramillo', 'Natalia Roja...","['Stonemaier Games', '999 Games', 'Angry Lion ..."
Viticulture Essential Edition,"['Economic', 'Farming']","['Contracts', 'Hand Management', 'Solo / Solit...","['Jamey Stegmaier', 'Alan Stone']","['Jacqui Davis', 'David Montgomery', 'Beth Sob...","['Stonemaier Games', 'Angry Lion Games', 'Arcl..."
Tapestry,['Civilization'],"['Area Majority / Influence', 'Card Drafting',...",['Jamey Stegmaier'],"['Andrew Bosley', 'Rom Brown']","['Stonemaier Games', '999 Games', 'Albi', 'Ang..."
Terraforming Mars,"['Economic', 'Environmental', 'Industry / Manu...","['Drafting', 'End Game Bonuses', 'Hand Managem...",['Jacob Fryxelius'],['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Brädspel.se', 'Fant..."


1. เกม Top 3 มี `publisher` ที่คล้ายกันหรือ `mechanic` ที่คล้ายกัน (`Solo / Solitaire Game`, `Card Drafting`, `Dice Rolling`)
1. `Wingspan` และ `Terraforming Mars` มี `publisher` เป็น `Siam Board Games` ซึ่งมีการแปลและผลิตเกมเหล่านี้เป็นภาษาไทย

In [31]:
df_rv_rate = df_rv.set_index('user').copy()
df_rv_rate = df_rv_rate.groupby(['user', 'name']).mean()
df_rv_rate

rating
user            name                                  
1 Family Meeple 10 Days in Europe                  4.1
                12 Days                            7.0
                7 Wonders                          6.5
                A Column of Fire                   5.0
                A Feast for Odin                  10.0
...                                                ...
zuzusdad        Wreck Raiders                      9.2
                XCOM: The Board Game               8.4
                Xia: Legends of a Drift System     6.5
                Zombie Dice                        7.0
                Zooloretto                         9.9

[2033403 rows x 1 columns]

In [32]:
# select 10 users
ten_users = list(df_rv_rate.index.get_level_values('user').unique()[:10])
ten_users

['1 Family Meeple',
 '1000rpm',
 '100pcBlade',
 '28green',
 '42amu',
 '549sd',
 'AHforever',
 'AJBrandon',
 'ASSaali',
 'Aarkas']

In [33]:
df_10users = df_rv_rate[df_rv_rate.index.get_level_values('user').isin(ten_users)].copy()
df_10users

rating
user            name                     
1 Family Meeple 10 Days in Europe     4.1
                12 Days               7.0
                7 Wonders             6.5
                A Column of Fire      5.0
                A Feast for Odin     10.0
...                                   ...
Aarkas          Xi'an                 7.0
                Yokohama              9.0
                Yspahan               8.0
                ZhanGuo               7.0
                Zombie Dice           7.0

[5638 rows x 1 columns]

In [34]:
for user in ten_users:
  # dataframe of favourite games
  df_user = df_10users.loc[user]
  avg = df_user['rating'].mean()
  df_fav = df_user[df_user['rating'] >= avg]  

  # most facourite game
  game_fav = df_fav['rating'].idxmax()

  # top 10 similar games
  df_sim = content_sim_df[game_fav].sort_values(ascending=False).head(10)

  # create lists to store the true and predicted labels
  true_labels = []
  pred_labels = []

  for game,sim in df_sim.iteritems():
      # if the recommendation is in the user's preferred games, it is a true positive
      if game in df_fav.index:
          true_labels.append(1)
          pred_labels.append(1)
      # if the recommendation is not in the user's preferred games, it is a false positive
      else:
          true_labels.append(0)
          pred_labels.append(1)

  # for each game in the user's preferred games, check if it was recommended
  for game,rate in df_fav.itertuples():
    if game not in df_sim:
      true_labels.append(1)
      pred_labels.append(0)

  # calculate precision and recall
  precision, recall, _, _ = precision_recall_fscore_support(y_true=true_labels, y_pred=pred_labels, average='binary')

  # print the results
  print('Precision:', precision)
  print('Recall:', recall)

Precision: 0.4
Recall: 0.013289036544850499
Precision: 0.4
Recall: 0.02127659574468085
Precision: 0.5
Recall: 0.0228310502283105
Precision: 0.2
Recall: 0.005917159763313609
Precision: 0.1
Recall: 0.003355704697986577
Precision: 0.5
Recall: 0.00998003992015968
Precision: 0.4
Recall: 0.017241379310344827
Precision: 0.4
Recall: 0.012121212121212121
Precision: 0.5
Recall: 0.017793594306049824
Precision: 0.1
Recall: 0.004807692307692308
